This notebook is used to test the evaluation metrics comes with  the MuspiPy library

In [1]:
!which python

/home/e20365/miniconda3/envs/muspy-env/bin/python


In [2]:
from pathlib import Path
import muspy

DATA_ROOT = "/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/data/raw/lmd_full"

midi_files = list(Path(DATA_ROOT).rglob("*.mid"))
len(midi_files)

178561

In [11]:
# sample data
music = muspy.read_midi(midi_files[3])
music


Music(metadata=Metadata(schema_version='0.2', title='None', source_filename='25bff980c03b216a59c478fc3c348856.mid', source_format='midi'), resolution=240, tempos=[Tempo(time=0, qpm=120.0), Tempo(time=104760, qpm=117.99990166674861), Tempo(time=104820, qpm=116.00008506672904), ...], key_signatures=[KeySignature(time=0, root=0, mode='major')], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], tracks=[Track(program=24, is_drum=False, name='Vocals    ', notes=[Note(time=8160, pitch=75, duration=90, velocity=63), Note(time=8280, pitch=73, duration=90, velocity=63), Note(time=8400, pitch=71, duration=90, velocity=63), ...], annotations=[Annotation(time=0, annotation={'number': 7, 'value': 108}, group='control_change'), Annotation(time=480, annotation={'number': 0, 'value': 0}, group='control_change'), Annotation(time=485, annotation={'number': 32, 'value': 0}, group='control_change'), ...]), Track(program=1, is_drum=False, name='Piano     ', notes=[Note(time=960, pitch=59,

## Inspect your MIDI (do this first)




To convert to an audio file can use this library as well. or net-seq?

## 1. Install FluidSynth

### Install via conda-forge

```bash
conda install -c conda-forge fluidsynth -y

```

### Verify (do not skip)

```bash
which fluidsynth
fluidsynth --version

```

# Get a soundfont (no archive.org, no sudo)

Do **NOT** rely on `muspy.download_musescore_soundfont()`

(it often fails on servers).

### Download MuseScore General SoundFont (GitHub mirror)

```bash
mkdir -p /scratch1/e20-fyp-xlstm-music-generation/assets/soundfonts
cd /scratch1/e20-fyp-xlstm-music-generation/assets/soundfonts

wget https://github.com/musescore/MuseScore/raw/master/share/sound/FluidR3Mono_GM.sf3

```

### Verify

```
ls -lh FluidR3Mono_GM.sf3
```

# Verify MusPy can see everything (sanity test)

Run **inside Python** (terminal or notebook):

```python
import shutil, os

print("fluidsynth:", shutil.which("fluidsynth"))
print("TMPDIR:", os.environ.get("TMPDIR"))

```

## Test `muspy.synthesize()`

```jsx
import muspy
import soundfile as sf

# Load any small MIDI file
music = muspy.read_midi(midi_files[3])

audio = muspy.synthesize(
    music,
    soundfont_path="/scratch1/e20-fyp-xlstm-music-generation/assets/soundfonts/FluidR3Mono_GM.sf3",
    rate=44100
)

sf.write("example.wav", audio, 44100)

```

## Find the current working directory (do this first)

In your Jupyter notebook, run **this cell**:

```python
import os
os.getcwd()

```

This prints the exact directory where `example.wav` was written.

In [14]:
import muspy
import numpy as np

# ================================
# Pitch-related metrics
# ================================

# 1. Pitch range
# Difference between highest and lowest pitch used (drum tracks ignored)
# Low -> narrow register, High -> wide expressive range
pitch_range = muspy.pitch_range(music)

# 2. Number of unique pitches used (absolute note variety)
n_pitches = muspy.n_pitches_used(music)

# 3. Number of unique pitch classes used (C, C#, D... ignoring octave)
# Range: 0–12, reflects harmonic palette
n_pitch_classes = muspy.n_pitch_classes_used(music)

# 4. Polyphony
# Average number of notes sounding simultaneously (only when notes are active)
polyphony = muspy.polyphony(music)

# 5. Polyphony rate
# Fraction of time steps where >= threshold notes are active
# Indicates how often chords occur
polyphony_rate = muspy.polyphony_rate(music, threshold=2)

# 6. Pitch-in-scale rate (example: C major)
# Fraction of notes that belong to a specific hypothesized key
pitch_in_c_major = muspy.pitch_in_scale_rate(
    music,
    root=0,          # C
    mode="major"
)

# 7. Scale consistency
# Maximum pitch-in-scale rate over all major/minor keys
# Key-agnostic tonal coherence
scale_consistency = muspy.scale_consistency(music)

# 8. Pitch entropy
# Entropy of pitch distribution (octave-sensitive)
# Low -> repetitive, High -> random
pitch_entropy = muspy.pitch_entropy(music)

# 9. Pitch class entropy
# Entropy of pitch class distribution (octave-invariant)
pitch_class_entropy = muspy.pitch_class_entropy(music)


# ================================
# Rhythm-related metrics
# ================================

# 10. Empty beat rate
# Fraction of beats with no notes at all
empty_beat_rate = muspy.empty_beat_rate(music)

# 11. Drum-in-pattern rate (duple meter example)
# Measures how well drum notes align with canonical drum patterns
drum_in_duple = muspy.drum_in_pattern_rate(
    music,
    meter="duple"
)

# 12. Drum pattern consistency
# Best alignment over duple and triple meters
drum_pattern_consistency = muspy.drum_pattern_consistency(music)

# 13. Groove consistency
# Mean Hamming distance between onset patterns of adjacent measures
# Requires constant time signature
# measure_resolution = time steps per measure (e.g., 4 beats * resolution)
measure_resolution = 4 * music.resolution
groove_consistency = muspy.groove_consistency(
    music,
    measure_resolution=measure_resolution
)


# ================================
# Other metrics
# ================================

# 14. Empty measure rate
# Fraction of entire measures with no notes
empty_measure_rate = muspy.empty_measure_rate(
    music,
    measure_resolution=measure_resolution
)


# ================================
# Collect and display results
# ================================

metrics = {
    "Pitch range": pitch_range,
    "Unique pitches": n_pitches,
    "Unique pitch classes": n_pitch_classes,
    "Polyphony": polyphony,
    "Polyphony rate": polyphony_rate,
    "Pitch-in-scale rate (C major)": pitch_in_c_major,
    "Scale consistency": scale_consistency,
    "Pitch entropy": pitch_entropy,
    "Pitch class entropy": pitch_class_entropy,
    "Empty beat rate": empty_beat_rate,
    "Drum-in-pattern rate (duple)": drum_in_duple,
    "Drum pattern consistency": drum_pattern_consistency,
    "Groove consistency": groove_consistency,
    "Empty measure rate": empty_measure_rate,
}

# Pretty print
for k, v in metrics.items():
    print(f"{k:35s}: {v}")


Pitch range                        : 53
Unique pitches                     : 36
Unique pitch classes               : 11
Polyphony                          : 4.766279239410592
Polyphony rate                     : 0.867474302496329
Pitch-in-scale rate (C major)      : 0.35693215339233036
Scale consistency                  : 0.9710914454277286
Pitch entropy                      : 4.215356009941102
Pitch class entropy                : 2.7269049895751016
Empty beat rate                    : 0.01098901098901095
Drum-in-pattern rate (duple)       : 1.0
Drum pattern consistency           : 1.0
Groove consistency                 : 0.9976216814159292
Empty measure rate                 : 0.00877192982456143
